# Ноутбук для подбора количества кофе, которое минимизирует стресс

## main class

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import gamma, norm


class HandsTable():
    minimize = True
    rho = 3.37

    def __init__(self, options_list, minimize=True, rho=1.0):
        """инициализация

        Args:
            options_list (list): список из названий
            minimize (bool, optional): Минимизировать если True, максимизировать если False. Defaults to True.
            rho (float, optional): Риск толерантность, чем больше, тем больше риска готовы принять. Defaults to 1.0.
        """
        self.hands = pd.DataFrame({'name': options_list,
                                   'mu': 0.0,
                                   'Te': 0.0,
                                   'alpha': 0.5,
                                   'beta': 0.5
                                   })
        self.minimize = minimize
        if rho is not None:
            self.rho = rho

        self.history = pd.DataFrame(columns=['option', 'value'])

    @classmethod
    def to_minutes(self, timestr: str):
        '''
        convert timestr to float minutes
        '''
        return pd.to_timedelta(timestr).total_seconds()/60

    @classmethod
    def update_mean(self, X, T_last, mu_last):
        mu_new = T_last * mu_last / (T_last + 1) + X / (T_last + 1)
        return mu_new

    @classmethod
    def update_samples(self, T):
        return T + 1

    @classmethod
    def update_shape(self, a):
        return a + 0.5

    @classmethod
    def update_rate(self, X, mu_last, beta_last, T_last):
        beta_new = beta_last + (T_last / (T_last + 1)) * \
            (np.square(X - mu_last)) / 2
        return beta_new

    def update_hands(self, name, value):
        if isinstance(value, str):
            try:
                value = HandsTable.to_minutes(value)
            except ValueError:
                raise ValueError('input time string in hh:mm:ss format')
        elif isinstance(value, float) or isinstance(value, int):
            pass
        else:
            raise ValueError('input time string or int/float value')

        _, mu, t, alpha, beta = self.hands[self.hands.name == name].values[0]
        beta = HandsTable.update_rate(value, mu, beta, t)
        mu = HandsTable.update_mean(value, t, mu)
        t = HandsTable.update_samples(t)
        alpha = HandsTable.update_shape(alpha)

        # added code to write history
        self.history.loc[len(self.history.index)] = [name, value]

        self.hands.loc[self.hands.name == name, 'mu'] = mu
        self.hands.loc[self.hands.name == name, 'Te'] = t
        self.hands.loc[self.hands.name == name, 'alpha'] = alpha
        self.hands.loc[self.hands.name == name, 'beta'] = beta

    def grade(self):
        hands_output = self.hands.copy()
        tau = gamma.rvs(a=hands_output.alpha, scale=1/hands_output.beta)
        theta_drops = norm.rvs(hands_output.mu, 1/hands_output.Te)
        hands_output['tau'] = tau
        hands_output['theta'] = theta_drops
        hands_output['SD'] = np.sqrt(1/tau)

        if self.minimize == True:
            hands_output['var95'] = theta_drops + \
                norm.ppf(1-0.05/2) * hands_output.SD
            if hands_output.mu.min() == 0:

                output_df = hands_output.reindex(np.argsort(hands_output.Te))
            else:
                output_df = hands_output.reindex(
                    np.argsort(self.rho * theta_drops + 1/tau))
        else:
            hands_output['var95'] = theta_drops + \
                norm.ppf(0.05/2) * hands_output.SD
            if hands_output.mu.min() == 0:
                output_df = hands_output.reindex(np.argsort(hands_output.Te))
            else:
                output_df = hands_output.reindex(
                    np.argsort(self.rho * theta_drops - 1/tau)[::-1])

        return output_df

    def __str__(self):
        return repr(self.hands)

# Применение

In [2]:
cups = HandsTable(options_list=[*range(6+1)], minimize=True)
cups.update_hands(0, 32) # суббота 9 ноября
cups.update_hands(2, 33) # воскресенье 10 ноября
cups.update_hands(1, 35) # вторник 12 ноября
cups.update_hands(3, 34) # среда 13 ноября
cups.update_hands(4, 34) # четверг 14 ноября
cups.update_hands(3, 34) # пятница 15 ноября
cups.update_hands(1, 34) # суббота 16 ноября
cups.update_hands(1, 43) # воскресенье 17 ноября
cups.update_hands(6, 28) # понедельник 18 ноября
# cups.update_hands(5, 38) # вторник 19 ноября

cups.grade()

,name,mu,Te,alpha,beta,tau,theta,SD,var95
Te,,,,,,,,,
5,5,0.000000,0.0,0.5,0.500000,0.635831,-inf,1.254091,-inf
0,0,32.000000,1.0,1.0,0.500000,3.903700,32.847754,0.506130,33.839750
2,2,33.000000,1.0,1.0,0.500000,2.757644,32.152853,0.602186,33.333116
4,4,34.000000,1.0,1.0,0.500000,0.215807,35.156687,2.152621,39.375746
6,6,28.000000,1.0,1.0,0.500000,1.646549,27.568908,0.779314,29.096336
3,3,34.000000,2.0,1.5,0.500000,1.007453,34.906981,0.996294,36.859682
1,1,37.333333,3.0,2.0,24.833333,0.035164,36.875626,5.332776,47.327676


# Какой выход из МЦК утром?
* Садиться в ту дверь, которая кажется ближе к выходу
* считать от выхода на платформу до выхода из крутящихся дверей офиса


In [3]:
morning_exit = HandsTable(options_list=[3,6], minimize=True)
# 3 мцк
# 6 мцд
morning_exit.update_hands(3, '0:08:03.7') # вторник 19 ноября

morning_exit.grade()

,name,mu,Te,alpha,beta,tau,theta,SD,var95
Te,,,,,,,,,
1,6,0.000000,0.0,0.5,0.5,0.026153,-inf,6.183583,-inf
0,3,8.061667,1.0,1.0,0.5,8.166267,9.646372,0.349936,10.332234


# Какой вход в МЦК вечером?
* Садиться в ту дверь, которая кажется ближе к выходу
* считать от выхода из крутящихся дверей до посадки в поезд


In [4]:
evening_entrance = HandsTable(options_list=[3,6], minimize=True)
# 3 мцк
# 6 мцд
evening_entrance.update_hands(6, '0:08:03.7') # четверг 21 ноября

evening_entrance.grade()

,name,mu,Te,alpha,beta,tau,theta,SD,var95
Te,,,,,,,,,
0,3,0.000000,0.0,0.5,0.5,0.387417,-inf,1.606611,-inf
1,6,8.061667,1.0,1.0,0.5,0.041370,8.291488,4.916497,17.927644
